In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '../')

# %matplotlib inline
# import matplotlib.pyplot as plt
# import numpy as np
from IPython.display import Markdown, display

# # Datasets
# from aif360.datasets import MEPSDataset19
# from aif360.datasets import MEPSDataset20
# from aif360.datasets import MEPSDataset21

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
# from aif360.metrics import ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# # Scalers
# from sklearn.preprocessing import StandardScaler

# # Classifiers
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline

# # Bias mitigation techniques
# from aif360.algorithms.preprocessing import Reweighing
# from aif360.algorithms.inprocessing import PrejudiceRemover

# # LIME
# from aif360.datasets.lime_encoder import LimeEncoder
# import lime
# from lime.lime_tabular import LimeTabularExplainer

# np.random.seed(1)

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [3]:
data = fetch_openml(data_id=1590, as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1

c:\Users\I2044\Anaconda3\envs\python38\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
# Train-Test-Val split of 5:3:2
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.5, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [10]:
from aif360.datasets import StandardDataset, BinaryLabelDataset

def create_standard_dataset(df: pd.DataFrame) -> StandardDataset:
    
    # races = ['race_Amer-Indian-Eskimo', 'race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White']
    protected_attribute_names = ['sex_Male']
    privileged_classes=[[1]]
    
    dataset = StandardDataset(
        df, 
        label_name='class', 
        favorable_classes=[1], 
        protected_attribute_names=protected_attribute_names, 
        privileged_classes=privileged_classes
    )

    return dataset

In [21]:
dataset_train = create_standard_dataset(pd.concat([X_train, y_train], axis=1))
dataset_test = create_standard_dataset(pd.concat([X_test, y_test], axis=1))
dataset_val = create_standard_dataset(pd.concat([X_val, y_val], axis=1))
dataset = create_standard_dataset(pd.concat([X, y_true], axis=1))

In [12]:
def describe(train=None, val=None, test=None):
    if train is not None:
        display(Markdown("#### Training Dataset shape"))
        print(train.features.shape)
    if val is not None:
        display(Markdown("#### Validation Dataset shape"))
        print(val.features.shape)
    display(Markdown("#### Test Dataset shape"))
    print(test.features.shape)
    display(Markdown("#### Favorable and unfavorable labels"))
    print(test.favorable_label, test.unfavorable_label)
    display(Markdown("#### Protected attribute names"))
    print(test.protected_attribute_names)
    display(Markdown("#### Privileged and unprivileged protected attribute values"))
    print(test.privileged_protected_attributes, 
          test.unprivileged_protected_attributes)
    display(Markdown("#### Dataset feature names"))
    print(test.feature_names)

In [13]:
describe(dataset_train, dataset_val, dataset_test)

#### Training Dataset shape

(24421, 105)


#### Validation Dataset shape

(12211, 105)


#### Test Dataset shape

...

['sex_Male']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass_Federal-gov', 'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc', 'workclass_State-gov', 'workclass_Without-pay', 'education_10th', 'education_11th', 'education_12th', 'education_1st-4th', 'education_5th-6th', 'education_7th-8th', 'education_9th', 'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate', 'education_HS-grad', 'education_Masters', 'education_Preschool', 'education_Prof-school', 'education_Some-college', 'marital-status_Divorced', 'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse', 'marital-status_Married-spouse-absent', 'marital-status_Never-married', 'marital-status_Separated', 'marital-status_Widowed', 'occupation_Adm-clerical', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occup

In [14]:
unprivileged_groups = [{'sex_Male': 0.0}]
privileged_groups = [{'sex_Male': 1.0}]

In [15]:
unprivileged_groups = [
    {k:v
    for index, k in enumerate(dataset_train.protected_attribute_names)
    for v in dataset_train.unprivileged_protected_attributes[index]}
]
privileged_groups = [
    {k:v
    for index, k in enumerate(dataset_train.protected_attribute_names)
    for v in dataset_train.privileged_protected_attributes[index]}
]

In [16]:
unprivileged_groups

[{'sex_Male': 0.0}]

In [17]:
privileged_groups

[{'sex_Male': 1.0}]

In [28]:
metric_train = BinaryLabelDatasetMetric(
        dataset_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
explainer_train = MetricTextExplainer(metric_train)

In [29]:
print(explainer_train.mean_difference())

Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.19797755075926476


In [30]:
print(explainer_train.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.3569394383883077


In [41]:
# print(explainer_train.consistency())

Consistency (Zemel, et al. 2013): [0.76366242]


In [42]:
print(explainer_train.statistical_parity_difference())

Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.06692745994099612


In [43]:
print(explainer_train.mean_difference())

Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.06692745994099612


In [67]:
from sklearn.tree import DecisionTreeClassifier


classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_train)

In [68]:
dataset_pred = dataset_train.copy()
dataset_pred.labels = y_pred

In [69]:
metric_val = ClassificationMetric(
        dataset_train,
        dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
explainer_val = MetricTextExplainer(metric_val)

In [70]:
explainer_val.disparate_impact()

'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.6263909604519774'

In [1]:
explainer_val.statistical_parity_difference()

NameError: name 'explainer_val' is not defined

In [ ]:
# Doc

In [ ]:
# Things to do
# 1. Check ClassificationMetric (model training will be required) and compare with BinaryLabelDatasetMetric
# 2. Check for Disparate-Impact scores for multiple 
# 3. Perform multiple mitigation algorithms for these sensitive features & privileged/unprovileged groups, and compare their DI


"""
Create a Python class to - 
    1. specify features on which to check ethical bias among a list of features
    2. check disparate impact on training data (preprocessing)
    3. highlight the DI, (along with some meaningful viz)
"""